In [ ]:
import pandas as pd
import numpy as np

# ▣ 1) 거리 계산 함수 정의 (Haversine 공식을 사용)
def haversine(lon1, lat1, lon2, lat2):
    """
    두 지점(위도 lat1, 위도 lat2, 경도 lon1, 경도 lon2) 사이의
    거리를 킬로미터 단위로 계산하여 반환하는 함수.
    """
    # 지구 반경 (단위: km)
    R = 6371.0
    
    # 라디안(radian) 단위로 변환
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    
    # Haversine 공식
    a = np.sin(delta_phi / 2.0) ** 2 + \
        np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    distance = R * c
    return distance

# ▣ 2) 기상관측 기기 목록 준비
# 사용자께서 알려주신 "대구북구 관측소"를 포함한 예시. 
# 실제 사용 시, 나머지 2개의 기기 정보(이름·위도·경도)를 동일한 형식으로 추가하십시오.
weather_stations = [
    {"name": "대구북구", "latitude": 35.90842, "longitude": 128.59068},
    {"name": "신암",     "latitude": 35.88515, "longitude": 128.61903},
    {"name": "달성",     "latitude": 35.69018, "longitude": 128.42384},
    {"name": "대구서구",   "latitude": 35.86509, "longitude": 128.53095},
    {"name": "옥포",     "latitude": 35.80279, "longitude": 128.44562},
    {"name": "하빈",     "latitude": 35.9062, "longitude": 128.44636},
]

# DataFrame 형태로도 변환 가능
weather_df = pd.DataFrame(weather_stations)
# └-> 컬럼: ['name', 'latitude', 'longitude']





In [ ]:
# ▣ 3) 지하철역 목록 준비
subway_df_1 = pd.read_csv('../지하철위도경도/대구교통공사_1호선 역별 출구별 위치정보_20240826.csv', encoding= 'cp949')
subway_df_2 = pd.read_csv('../지하철위도경도/대구교통공사_2호선 역별 출구별 위치정보_20241014.csv', encoding= 'cp949')
subway_df_3 = pd.read_csv('../지하철위도경도/대구교통공사_3호선 역별 출구별 위치정보_20240826.csv', encoding= 'cp949')

In [35]:
subway_df = pd.concat([subway_df_1, subway_df_2 ,subway_df_3] , axis = 0,ignore_index=True)


subway_df = subway_df.rename(columns={'위도' : 'latitude' , '경도' :'longitude'})

In [36]:
subway_df.tail(10)

,호선,역명,출구번호,latitude,longitude
419,3,수성못,1,35.83179,128.6233
420,3,지산,1,35.82541,128.6316
421,3,범물,1,35.82110,128.6400
422,3,범물,2,35.82092,128.6398
423,3,범물,3,35.82071,128.6401
424,3,범물,4,35.82090,128.6403
425,3,용지,1,35.81847,128.6465
426,3,용지,2,35.81809,128.6466
427,3,용지,3,35.81815,128.6471
428,3,용지,4,35.81854,128.6470


In [37]:

# ▣ 4) 각 지하철역마다 가장 가까운 기상관측 기기를 찾아서 매핑
#    결과를 저장할 빈 리스트 초기화
nearest_station_list = []
nearest_distance_list = []

# 반복문을 사용해 각 지하철역에 대해 수행
for idx, row in subway_df.iterrows():
    station_lat = row["latitude"]
    station_lon = row["longitude"]
    
    # 이 역에서 모든 기상 기기까지의 거리를 계산 → 리스트로 저장
    distances = []
    for _, w_row in weather_df.iterrows():
        w_lat = w_row["latitude"]
        w_lon = w_row["longitude"]
        dist_km = haversine(station_lon, station_lat, w_lon, w_lat)
        distances.append(dist_km)
    
    # 가장 작은 거리의 인덱스를 찾는다
    min_idx = np.argmin(distances)
    min_distance = distances[min_idx]
    min_station_name = weather_df.loc[min_idx, "name"]
    
    # 결과 리스트에 추가
    nearest_station_list.append(min_station_name)
    nearest_distance_list.append(min_distance)

# ▣ 5) 결과를 원본 subway_df에 컬럼으로 추가하기
subway_df["nearest_weather_station"] = nearest_station_list
subway_df["distance_km_to_weather"] = nearest_distance_list

# 최종 확인
print(subway_df)

     호선    역명  출구번호  latitude  longitude nearest_weather_station  \
0     1  설화명곡     1  35.79898   128.4888                      옥포   
1     1  설화명곡     2  35.79848   128.4888                      옥포   
2     1  설화명곡     3  35.79833   128.4890                      옥포   
3     1  설화명곡     4  35.79831   128.4896                      옥포   
4     1  설화명곡     5  35.79868   128.4901                      옥포   
..   ..   ...   ...       ...        ...                     ...   
424   3    범물     4  35.82090   128.6403                      신암   
425   3    용지     1  35.81847   128.6465                      신암   
426   3    용지     2  35.81809   128.6466                      신암   
427   3    용지     3  35.81815   128.6471                      신암   
428   3    용지     4  35.81854   128.6470                      신암   

     distance_km_to_weather  
0                  3.917173  
1                  3.923587  
2                  3.943556  
3                  3.997521  
4                  4.037393  
.. 

In [38]:
subway_df.head(100)

,호선,역명,출구번호,latitude,longitude,nearest_weather_station,distance_km_to_weather
0,1,설화명곡,1,35.79898,128.4888,옥포,3.917173
1,1,설화명곡,2,35.79848,128.4888,옥포,3.923587
2,1,설화명곡,3,35.79833,128.4890,옥포,3.943556
3,1,설화명곡,4,35.79831,128.4896,옥포,3.997521
4,1,설화명곡,5,35.79868,128.4901,옥포,4.037393
...,...,...,...,...,...,...,...
95,1,반월당1,23,35.86478,128.5932,신암,3.247580
96,1,중앙로,1,35.86987,128.5937,신암,2.845201
97,1,중앙로,2,35.87002,128.5939,신암,2.820786
98,1,중앙로,3,35.87198,128.5943,신암,2.666269


In [47]:
# 각 역명별로 nearest_weather_station의 고유 개수 계산
station_counts = subway_df.groupby('역명')['nearest_weather_station'] \
                   .nunique() \
                   .reset_index(name='unique_count')

print(station_counts)

      역명  unique_count
0     각산             1
1     감삼             1
2     강창             1
3   건들바위             1
4   경대병원             1
..   ...           ...
86    학정             1
87    해안             1
88   현충로             1
89    화원             1
90    황금             1

[91 rows x 2 columns]


In [ ]:
# unique_count > 1인 역명 리스트
problematic_stations = station_counts[
    station_counts['unique_count'] > 1
]['역명'].tolist()
print(problematic_stations)  

[]


In [49]:
subway_df.to_excel('역_동매핑.xlsx', index = False)